In [21]:
#from google.colab import drive
import glob
#drive.mount('/content/drive')


In [22]:
# IMPORT NECESSARY LIBRARIES
import numpy as np
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
import pandas as pd
from sklearn.metrics import confusion_matrix
import sys
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import itertools
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Dense, Embedding, LSTM
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from sklearn.utils import shuffle
from tensorflow.python.keras import layers, models, Model, optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import layers, models, Model, optimizers
from random import randint
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from matplotlib import pyplot



In [23]:
angry = glob.glob('data_images/angry/*.*')
calm = glob.glob('data_images/calm/*.*')
disgust = glob.glob('data_images/disgust/*.*')
fearful = glob.glob('data_images/fearful/*.*')
happy = glob.glob('data_images/happy/*.*')
neutral = glob.glob('data_images/neutral/*.*')
sad = glob.glob('data_images/sad/*.*')
surprised = glob.glob('data_images/surprised/*.*')

In [26]:
data = []
labels = []

for i in angry:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Angry')
for i in calm:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Calm')
for i in disgust:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Disgust')
for i in fearful:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Fearful')
for i in happy:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Happy')
for i in neutral:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Neutral')
for i in sad:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Sad')
for i in surprised:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('Surprised')

data = np.array(data)
labels = np.array(labels)
print(labels)

[]


In [25]:
encoder = OneHotEncoder()
labels = encoder.fit_transform(np.array(labels).reshape(-1,1)).toarray()

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=0, test_size=0.3, shuffle=True)

In [7]:
print(X_train.shape)
print(X_test.shape)

(1008, 224, 224, 3)
(432, 224, 224, 3)


In [8]:
print(y_train.shape)
print(y_test.shape)

(1008, 8)
(432, 8)


In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/ 255
X_test = X_test/ 255
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1008, 224, 224, 3), (1008, 8), (432, 224, 224, 3), (432, 8))

In [14]:
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
cnn = keras.Sequential()
cnn.add(Conv2D(128,(5,5),activation='relu',input_shape=(224,224,3)))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(64,(5,5),activation='relu',kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Dropout(0.3))
cnn.add(Conv2D(32,(5,5),activation='relu'))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Flatten())
cnn.add(Dense(64,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(8,activation='softmax'))
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 220, 220, 128)     9728      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 110, 110, 128)    0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 110, 110, 128)     0         
                                                                 
 conv2d_7 (Conv2D)           (None, 106, 106, 64)      204864    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 53, 53, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 53, 53, 64)       

In [ ]:
cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = cnn.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
32/32 [==============================] - 17s 363ms/step - loss: 2.2365 - accuracy: 0.1458 - val_loss: 2.1178 - val_accuracy: 0.1574
Epoch 2/50
32/32 [==============================] - 9s 286ms/step - loss: 2.0922 - accuracy: 0.1746 - val_loss: 2.0961 - val_accuracy: 0.2639
Epoch 3/50
32/32 [==============================] - 9s 296ms/step - loss: 2.0511 - accuracy: 0.1915 - val_loss: 2.0190 - val_accuracy: 0.2477
Epoch 4/50
32/32 [==============================] - 9s 287ms/step - loss: 2.0101 - accuracy: 0.2242 - val_loss: 1.9703 - val_accuracy: 0.3218
Epoch 5/50
32/32 [==============================] - 9s 295ms/step - loss: 1.9649 - accuracy: 0.2272 - val_loss: 1.9293 - val_accuracy: 0.2894
Epoch 6/50
32/32 [==============================] - 9s 286ms/step - loss: 1.9386 - accuracy: 0.2649 - val_loss: 1.8540 - val_accuracy: 0.3171
Epoch 7/50
32/32 [==============================] - 9s 287ms/step - loss: 1.8524 - accuracy: 0.3046 - val_loss: 1.8266 - val_accuracy: 0.3843
Epoch